In [ ]:
import numpy as np
import pandas as pd
from scipy.io.wavfile import read 
import os


In [ ]:
# Storing the address in 'url' variable
data_path =   "dsl_data/development.csv"
# Importing the file from the address contained in 'url' into 'df' 
df = pd.read_csv(data_path)
df

In [ ]:
# Check null values in Training dataset (pandas dataframe)
df.isnull().sum().sum()

In [ ]:
# To get unique values in order to map values to objects imported from src/data.py

df['object'].unique()
df['action'].unique()


In [ ]:
import dict_data 

df['gender'] = df['gender'].map(dict_data.gender_map)
df['Self-reported fluency level '] = df['Self-reported fluency level '].map(dict_data.language_fluency_map)
df['ageRange'] = df['ageRange'].map(dict_data.age_range_map)
df['Current language used for work/school'] = df['Current language used for work/school'].map(dict_data.current_language_map)
df['First Language spoken'] = df['First Language spoken'].map(dict_data.first_language_map)


In [ ]:
path = os.getcwd() + "/"
audios_array = []
for audio in df['path']:
    temp_rate, temp_data = read(audio, mmap=False)    
    audios_array.append(temp_data)

df['audio'] = audios_array

    

In [ ]:
from sklearn.model_selection import train_test_split

x = df[['Self-reported fluency level ','First Language spoken', 'Current language used for work/school', 'gender', 'ageRange', 'audio']].copy()
y = df[['action', 'object']].copy()


#train_test_split splits the data into 75% training data and 25% test data
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Classifier
clf=RandomForestClassifier(n_estimators=10)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)
print("FIT")
y_pred=clf.predict(x_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))